In [12]:
# Use inline backend for static figures in VS Code
%matplotlib qt

import numpy as np
from tqdm import tqdm
import networkx as nx
import buildgraph

# Remove explicit backend selection to avoid conflicts
# import matplotlib
# matplotlib.use('macosx')

import matplotlib.pyplot as plt

print('Matplotlib backend:', plt.get_backend())

Matplotlib backend: qtagg


Hénon model: 
\begin{align}
    x_{n+1} &= y_n + 1 - ax_n^2\\
    y_{n+1} &= bx_n
\end{align}


In [155]:
def henon_map(a, b, x0, y0, n):
    """
    Generate points of the Henon map
    """
    x, y = x0, y0
    points = np.zeros((n, 2))
    for i in range(n):
        x_new = 1 - a * x**2 + y
        y_new = b * x
        points[i, :] = [x_new, y_new]
        x, y = x_new, y_new
    return points


def plot_henon_map(a, b, x0, y0, n, transitoire):
    """
    Plot the Henon map
    """
    points = henon_map(a, b, x0, y0, n)
    fig, ax = plt.subplots()
    ax.scatter(points[transitoire:, 0], points[transitoire:, 1])
    ax.set_title(f"Henon Map (a={a}, b={b})")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.axis("equal")
    ax.grid(True)
    plt.show()


def plot_bifurcation_diagram(a_values, b, x0, y0, n, last, axis):
    """
    Plot the bifurcation diagram of the Henon map
    """
    fig, ax = plt.subplots()
    if axis == "x":
        for a in a_values:
            points = henon_map(a, b, x0, y0, n)
            ax.scatter([a] * last, points[-last:, 0], s=0.5, c="blue")
    else:
        for a in a_values:
            points = henon_map(a, b, x0, y0, n)
            ax.scatter([a] * last, points[-last:, 1], s=0.5, c="blue")
    ax.set_title(f"Bifurcation Diagram (b={b})")
    ax.set_xlabel("a")
    ax.set_ylabel("x")
    ax.grid(True)
    plt.show()

In [3]:
plot_henon_map(a=1, b=0.3, x0=0.1, y0=0.1, n=100000, transitoire=100)

In [141]:
plot_bifurcation_diagram(a_values=np.linspace(0, 1.6, 1000), b=0.3, x0=0.1, y0=0.1, n=10000, last=100, axis="x")

/var/folders/g9/2s_b0h716cdgj18bgy7tjlww0000gn/T/ipykernel_7624/3436512866.py:8: RuntimeWarning: overflow encountered in scalar power
  x_new = 1 - a * x**2 + y


In [156]:
def coupled_henon_map(a, b, c, d, x0, y0, u0, v0, n, Eps):
    """Generate points of the coupled Henon map."""
    x, y = x0, y0
    u, v = u0, v0
    points = np.zeros((n, 2, 2))  # [(x, y, u, v)]
    for i in range(n):
        x_new = 1 - a * ((1 - Eps) * x + Eps * u) ** 2 + ((1 - Eps) * y + Eps * v)
        y_new = b * ((1 - Eps) * x + Eps * u)
        u_new = 1 - c * ((1 - Eps) * u + Eps * x) ** 2 + ((1 - Eps) * v + Eps * y)
        v_new = d * ((1 - Eps) * u + Eps * x)

        points[i, :, :] = [
            [x_new, y_new],
            [u_new, v_new],
        ]  # append((x_new, y_new, u_new, v_new))
        x, y = x_new, y_new
        u, v = u_new, v_new

    return points


def plot_coupled_henon_map(a, b, c, d, x0, y0, u0, v0, n, Eps, transitoire):
    """Plot the coupled Henon map."""
    points = coupled_henon_map(a, b, c, d, x0, y0, u0, v0, n, Eps)
    plt.plot(
        points[transitoire:, 0, 0],
        points[transitoire:, 0, 1],
        "b.",
        markersize=5,
        label="Map 1",
    )
    plt.plot(
        points[transitoire:, 1, 0],
        points[transitoire:, 1, 1],
        "r.",
        markersize=5,
        label="Map 2",
    )
    plt.title(f"Coupled Henon Map (a={a}, b={b}, c={c}, d={d}, Eps={Eps})")
    plt.xlabel("First coordinate")
    plt.ylabel("Second coordinate")
    plt.axis("equal")
    plt.grid(True)
    plt.legend()
    plt.show()

In [157]:
plot_coupled_henon_map(a=0.6, b=0.3,c = 1.2, d = 0.3, x0=0.1, y0=0.1, u0=0.1, v0=0.1, n=100000, Eps=0.1, transitoire=100) 

In [158]:
a = 1.2
b = 0.3
c = 0.3
d = 0.3
x0 = 0.0
y0 = 0.0
u0 = 0.0
v0 = 0.0
n = int(1e5)
points_gardes = 2000
transitoire = 500
Eps = [0.1, 0.2, 0.3, 0.4, 0.5]

In [159]:
def coupling_sum(x, y, delta):
    """Return the average sum wrt a certain coupling value"""
    assert 0 < delta < 1, "wrong coupling"
    return (1 - delta) * x + delta * y


def coupled_henon_wrt_coupling(a, b, c, d, x0, y0, u0, v0, n, Eps, points_gardes):
    """Compute the coupled Henon maps for several coupling strengths Eps.
    Returns an array of shape (points_gardes, 2, num_couplings) where axis 0=time, axis1=(x,y), axis2=coupling index

    be aware that it returns the coupled variables
    """
    p = len(Eps)
    points = np.zeros((points_gardes, 2, p))
    all_points = np.zeros((points_gardes, 2, 2, p))
    for i in range(p):
        all_points[:, :, :, i] = coupled_henon_map(
            a, b, c, d, x0, y0, u0, v0, n, Eps[i]
        )[-points_gardes:, :, :]
        points = coupling_sum(all_points[:, 0, :, :], all_points[:, 1, :, :], Eps[i])
    return points

In [160]:
simu = coupled_henon_wrt_coupling(a, b, c, d, x0, y0, u0, v0, n, Eps, points_gardes)
print('simu shape:', np.shape(simu))

simu shape: (2000, 2, 5)


In [163]:
fig, ax = plt.subplots()
for i in range(len(Eps)):
    ax.scatter(simu[:,0,i], simu[:,1,i], s=0.5, label=f"Eps = {Eps[i]}")
ax.set_xlabel("x")
ax.set_ylabel("y")
plt.legend()
plt.show()

détecter un régime périodique ou chaotique : pour un régime périodique le nombre de points stables différents est un multiple de 2

In [164]:
def count(data, axis: str):
    assert len(np.shape(data)) == 4, "wrong data shape input"
    if axis == "x":
        p = (0, 0)
    elif axis == "y":
        p = (0, 1)
    elif axis == "u":
        p = (1, 0)
    elif axis == "v":
        p = (1, 1)
    else:
        return "axis not existent"
    simu = np.sort(data[:, p[0], p[1], :], axis=0)
    k = len(simu[0])
    nb_points = [1] * k
    for i in range(k):
        start = simu[0, i]
        for j in range(1, len(simu)):
            if simu[j, i] != start:
                nb_points[i] += 1
                start = simu[j, i]
    return nb_points

In [165]:
def full_points(a, b, c, d, x0, y0, u0, v0, n, Eps, points_gardes):
    p = len(Eps)
    all_points = np.zeros((points_gardes, 2, 2, p))
    for i in range(p):
        all_points[:, :, :, i] = coupled_henon_map(
            a, b, c, d, x0, y0, u0, v0, n, Eps[i]
        )[-points_gardes:, :, :]
    return all_points

In [166]:
full_data = full_points(a, b, c, d, x0, y0, u0, v0, n, Eps, points_gardes)

Pour savoir si nombre de période discret ou continu il suffit de compter selon une des deux dimensions

In [19]:
rng = np.random.default_rng(12345)

In [20]:
Epsilon = rng.uniform(0, 1, 100)

In [168]:
Epsilon = np.abs(rng.normal(0.15, 1e-2, 1000))

In [169]:
data = full_points(a, b, c, d, x0, y0, u0, v0, n, Epsilon, points_gardes)
u_counting = count(data, "u")
x_counting = count(data, "x")
y_counting = count(data, "y")
v_counting = count(data, "v")

uncoupled data to compare with the coupling

In [170]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.scatter(Epsilon, x_counting, label="node 1: x", marker="x", s=10)
ax1.scatter(Epsilon, u_counting, label="node 2: u", marker="*", s=10)
ax1.set_xlabel("e coupling constant")
ax1.set_ylabel("number of different stable points of each node")
ax2.scatter(Epsilon, y_counting, label="node 1: y", marker="x", s=10)
ax2.scatter(Epsilon, v_counting, label="node 2: v", marker="*", s=10)
ax2.set_xlabel("e coupling constant")
ax1.legend()
ax2.legend()
plt.title("Fixed set of parameters for the Henon equations")
plt.show()

Création d'une fonction qui transforme un graphe en système de Hénons couplés


D'abord quelques fonctions pour transformer un graphe en Matrice de conectivité

In [171]:
def connectivty(G): 
    return nx.to_numpy_array(G)

#nx.to_scipy_sparse_array( ,format='csr')

def neighbors(A, i):
    neighborhood = []
    for j in range(len(A)):
        if A[j, i] == 1.0:
            neighborhood.append(j)
    return neighborhood

les fonctions pour l'intégration

In [185]:
def transfo_coupling_vec(G, X, Y, Eps):
    # X_new = np.zeros(len(G.nodes))
    # Y_new = np.zeros(len(G.nodes))
    m, n = len(X), len(Y)
    assert m == n, "missmatch dimension"
    Adjacance = connectivty(G)
    weights = np.sum(Adjacance, axis=1)
    if (weights == 0).any():
        raise ValueError(
            "some nodes have no edges/neighbors/connections with other nodes"
        )  # comme ça pas de problème de type sur la sortie
    transition = Eps * Adjacance * 1 / weights[:, np.newaxis] + np.identity(m)
    X_new = np.dot(transition, X) - Eps * X
    Y_new = np.dot(transition, Y) - Eps * Y
    return X_new, Y_new


def evolution_vec(G, X_0, Y_0, N, list_ab, Eps, Norm: bool, alpha=1.0):
    n, m = len(X_0), len(Y_0)
    assert n == m, "wrong dimensions, both initial conditions don't have the same size"
    X = np.zeros((n, N))
    Y = np.zeros((n, N))
    UN = np.ones(len(G.nodes))
    if Norm:
        X[:, 0] = X_0 / np.linalg.norm(X_0) / alpha
        Y[:, 0] = Y_0 / np.linalg.norm(Y_0) / alpha
        X_c, Y_c = transfo_coupling_vec(G, X[:, 0], Y[:, 0], Eps)
        for t in range(1, N):
            # print("evolution step", t)
            mediumX = -list_ab[0, :] * X_c**2 + UN + Y_c
            mediumY = list_ab[1, :] * X_c
            X[:, t] = mediumX / np.linalg.norm(mediumX) / alpha
            Y[:, t] = mediumY / np.linalg.norm(mediumY) / alpha
            X_c, Y_c = transfo_coupling_vec(G, X[:, t], Y[:, t], Eps)
        #print("compute dynamic done")
        return X, Y
    else:
        X[:, 0] = X_0
        Y[:, 0] = Y_0
        X_c, Y_c = transfo_coupling_vec(G, X[:, 0], Y[:, 0], Eps)
        for t in range(1, N):
            # print("evolution step", t)
            X[:, t] = -list_ab[0, :] * X_c**2 + UN + Y_c
            Y[:, t] = list_ab[1, :] * X_c
            X_c, Y_c = transfo_coupling_vec(G, X[:, t], Y[:, t], Eps)
        #print("compute dynamic done")
        return X, Y

there is an issue of we want to normalize againt time evolution because values are extremely small but not zero therefore if values are divided by the norm it leads to an overflow issue at some point

Exemple sur un graphe simple

In [3]:
rng = np.random.default_rng(1234)

In [ ]:
Number_Nodes = 2
p = rng.uniform(0, 1)
print(p)

0.9766997666981422


In [175]:
G = nx.erdos_renyi_graph(Number_Nodes, 1, seed=42)

In [176]:
plt.figure()
nx.draw(G, with_labels=True)
plt.show()

In [177]:
list_ab = np.zeros((2, Number_Nodes))
for i in range(Number_Nodes):
    list_ab[0, i] = rng.uniform(1.0, 1.2)
    list_ab[1, i] = 0.3

In [179]:
list_ab[1,:] = 0.3
list_ab[0,0] = 1.2
list_ab[0,1] = 0.3
list_ab

array([[1.2, 0.3],
       [0.3, 0.3]])

Application sur le graphe de la cellule ci-dessus

In [2]:
N = 10000
Eps = 0.15
iteration = np.arange(0, N, 1)
len(iteration)

10000

In [181]:
X_0 = [0.5 for _ in range(Number_Nodes)]
Y_0 = [0.5 for _ in range(Number_Nodes)]

TEST IF vectorized == not vectorized

In [102]:
X_vec, Y_vec = evolution_vec(G, X_0, Y_0, N, list_ab, Eps, True, alpha = 0.5)
X_not_vec, Y_not_vec = evolution_vec(G, X_0, Y_0, N, list_ab, Eps, False)

X_vec_array = X_vec.T
Y_vec_array = Y_vec.T

X_not_vec_array= X_not_vec.T
Y_not_vec_array= Y_not_vec.T

In [103]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))


colors = plt.colormaps['viridis'](np.linspace(0, 1, len(G.nodes)))
labels = [f"Nœud {i}" for i in G.nodes()]

# Tracé de X normé
for i in range(Number_Nodes):
    ax1.plot(X_vec_array[:, i], color=colors[i], label=labels[i])
ax1.set_title("Évolution de X normalisé")
ax1.set_xlabel("Itération")
ax1.set_ylabel("Valeur de X")
#ax1.legend()
ax1.grid(True)

# Tracé de X pas normé
for i in range(Number_Nodes):
    ax2.scatter(iteration, X_not_vec_array[:, i], color=colors[i], label=labels[i])
ax2.set_title("Évolution de X")
ax2.set_xlabel("Itération")
ax2.set_ylabel("Valeur de X")
#ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

In [104]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))


colors = plt.colormaps['viridis'](np.linspace(0, 1, len(G.nodes)))
labels = [f"Nœud {i}" for i in G.nodes()]

# Tracé de X
for i in range(Number_Nodes):
    ax1.plot(X_vec_array[:, i], color=colors[i], label=labels[i])
ax1.set_title("Évolution de X")
ax1.set_xlabel("Itération")
ax1.set_ylabel("Valeur de X")
#ax1.legend()
ax1.grid(True)

# Tracé de Y
for i in range(Number_Nodes):
    ax2.plot(Y_vec_array[:, i], color=colors[i], label=labels[i])
ax2.set_title("Évolution de Y")
ax2.set_xlabel("Itération")
ax2.set_ylabel("Valeur de Y")
#ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

In [82]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))


colors = plt.colormaps['viridis'](np.linspace(0, 1, len(G.nodes)))
labels = [f"Nœud {i}" for i in G.nodes()]

# Tracé de X
for i in range(Number_Nodes):
    ax1.plot(X_not_vec_array[:, i], color=colors[i], label=labels[i])
ax1.set_title("Évolution de X")
ax1.set_xlabel("Itération")
ax1.set_ylabel("Valeur de X")
#ax1.legend()
ax1.grid(True)

# Tracé de Y
for i in range(Number_Nodes):
    ax2.plot(Y_not_vec_array[:, i], color=colors[i], label=labels[i])
ax2.set_title("Évolution de Y")
ax2.set_xlabel("Itération")
ax2.set_ylabel("Valeur de Y")
#ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

In [182]:
def count2D(data, node):
    """
    warning data is either x data or y data
    """
    assert len(np.shape(data)) == 2, "wrong data shape input"
    simu = np.sort(data[node, :])
    k = len(simu)
    nb_points = 1
    start = simu[0]
    for i in range(k):
        if simu[i] != start:
            nb_points += 1
            start = simu[i]
    return nb_points

In [190]:
Epsilon = np.linspace(0.12, 0.18, 500)
X_0 = [0.5 for _ in range(Number_Nodes)]
Y_0 = [0.5 for _ in range(Number_Nodes)]
transitoire = 1000
node = 0

In [192]:
Xevol_forward, Yevol_forward = (
    np.zeros((len(X_0), N, len(Epsilon))),
    np.zeros((len(X_0), N, len(Epsilon))),
)
Xevol_backward, Yevol_backward = (
    np.zeros((len(X_0), N, len(Epsilon))),
    np.zeros((len(X_0), N, len(Epsilon))),
)

for i in tqdm(range(len(Epsilon))):
    Xevol_forward[:, :, i], Yevol_forward[:, :, i] = evolution_vec(
        G, X_0, Y_0, N, list_ab, Epsilon[i], False
    )

for i in tqdm(range(len(Epsilon))):
    X_f, Y_f = Xevol_forward[:, -1, i], Yevol_forward[:, -1, i]
    Xevol_backward[:, :, i], Yevol_backward[:, :, i] = evolution_vec(
        G, X_f, Y_f, N, list_ab, Epsilon[i], False
    )

100%|██████████| 500/500 [01:21<00:00,  6.14it/s]


In [193]:
count_forward = np.zeros(len(Epsilon))
for j in range(len(Epsilon)):
    count_forward[j] = count2D(Xevol_forward[:, transitoire:, j], node)

count_backward = np.zeros(len(Epsilon))
for j in range(len(Epsilon)):
    count_backward[j] = count2D(Xevol_backward[:, transitoire:, j], node)

In [194]:
fig, ax = plt.subplots()
ax.scatter(Epsilon, count_forward, s=5)
ax.scatter(Epsilon, count_backward, s=5)
plt.show()

In [14]:
from importlib import reload
reload(buildgraph)

<module 'buildgraph' from '/Users/romanbeauvallet/Documents/CoursM2ENS/Sys_Complex/Numerical_Project/buildgraph.py'>

In [ ]:
N = 3
std = 1
mean = 0.0
(xmax, ymax) = (1.0, 1.0)
test_pos = buildgraph.pos_nodes_uniform(N, xmax, ymax, rng)
Adja = buildgraph.connexion_normal_deterministic(test_pos, rng, std)
graphe = nx.from_numpy_array(Adja)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(test_pos[0, :], test_pos[1, :], s=1)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Nodes positions in space")
plt.show()

In [23]:
plt.figure()
nx.draw(graphe, with_labels=True)
plt.show()